In [1]:
import os 

from dotenv import load_dotenv
load_dotenv()


OLLAMA_URL=os.getenv("OLLAMA_URL")
LLM_MODEL=os.getenv("LLM_MODEL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
DATAPATH=os.getenv("DATAPATH")
#OLLAMA_URL='http://localhost:11434'
LLM_MODEL="llama3.2:1b"
#LLM_MODEL="llama3.2:3b" # max retires
#LLM_MODEL="gemma3:4b" # no tool support
#LLM_MODEL="llava:7b"
#LLM_MODEL="codellama:7b"
#LLM_MODEL="qwen2.5-coder:1.5b" # no tool support
#LLM_MODEL="deepseek-r1:1.5b" # does not support tools
#os.setenv("no_proxy=localhost,127.0.0.1,*.my-it-solutions.net,ollama")

print(OLLAMA_URL)
print(f"model: {LLM_MODEL}")
print(f"embedding: {EMBEDDING_MODEL}")
print(f"datapath {DATAPATH}")
print(f"no_proxy {os.getenv('no_proxy')}")

import requests
import time

response = requests.get(OLLAMA_URL,timeout=3)
if response.status_code == requests.codes.ok:
    print(response)
else:
    print(response)    

http://ollama-service-internal.default.svc.cluster.local:11433
model: llama3.2:1b
embedding: bge-m3:567m
datapath /app/data
no_proxy None
<Response [200]>


In [2]:
#from llama_index.llms.ollama import Ollama
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
#llm = Ollama(model=LLM_MODEL, request_timeout=120.0, base_url=OLLAMA_URL, temperature=0.0)


ollama_model = OpenAIModel(
    model_name='LLM_MODEL', provider=OpenAIProvider(base_url=f"{OLLAMA_URL}/v1")
)


In [18]:
from mcp.server.fastmcp import FastMCP

from starlette.applications import Starlette
from starlette.routing import Mount, Host
import uvicorn

from pydantic_ai import Agent
import asyncio
import nest_asyncio
nest_asyncio.apply()
#loop = asyncio.get_event_loop()


server = FastMCP('PydanticAI Server')
server_agent = Agent(model=ollama_model,
              result_type=str,
              deps_type=str,
              system_prompt="",              
              retries=2)

@server.tool()
async def poet(theme: str) -> str:
    """Poem generator"""
    print("test")
    r = await server_agent.run(f'write a poem about {theme}')    
    return r.data

@server.prompt()
def review_code(code: str) -> str:
    return f"Please review this code:\n\n{code}"    


#server.run()
#asyncio.run_coroutine_threadsafe(server.run('sse'), loop)
#asyncio.run(server.run('sse'))
# Mount the SSE server to the existing ASGI server
app = Starlette(
    routes=[
        Mount('/', app=server.sse_app()),
    ]
)

# or dynamically mount as host
app.router.routes.append(Host('localhost', app=server.sse_app()))

#LLM_MODEL="llama3.2:3b.run(app, host="127.0.0.1", port=8000)
bg = None
async def start_mcp_server(bg):
    bg = await uvicorn.run(app, host='0.0.0.0', port=8001, log_level="info")
    #config = uvicorn.Config("notebook:app", host='0.0.0.0', port=8001, log_level="info")
    #server = uvicorn.Server(config)
    #await server.serve()

if bg is not None:
    print("alrady runnign, stop")
    bg.stop()

asyncio.create_task(start_mcp_server(bg))
#start_mcp_server(bg)
print("started")

started


INFO:     Started server process [1667]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:55168 - "GET /sse HTTP/1.1" 200 OK


In [19]:
print("test")

test


In [ ]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerHTTP

server = MCPServerHTTP(url='http://localhost:8001/sse')  
agent = Agent(model=ollama_model,
              result_type=str,
              deps_type=str,
              system_prompt="",
              mcp_servers=[server],
              retries=1)


async def main():
    async with agent.run_mcp_servers():  
        result = await agent.run('Schreibe ein Gedicht über Füsse')
    print(result.data)
    #> There are 9,208 days between January 1, 2000, and March 18, 2025.

result = await main()
print(result.data)

[03/27/25 23:38:59] INFO     Connecting to SSE endpoint: http://localhost:8001/sse                        ]8;id=441288;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=691221;file:///opt/venv/lib/python3.13/site-packages/mcp/client/sse.py#45\45]8;;\

                    INFO     HTTP Request: GET http://localhost:8001/sse "HTTP/1.1 200 OK"          ]8;id=568873;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=635721;file:///opt/venv/lib/python3.13/site-packages/httpx/_client.py#1786\1786]8;;\